In [90]:
import pandas as pd
from pathlib import Path
import json

# Please download the bulk all data FAOSTAT dataset manually from:
# https://www.fao.org/faostat/en/#data/FBS
# Place the CSV file in the ../data/ directory relative to this script.

csv_path = Path('..') / 'data' / 'SUA_Crops_Livestock_E_All_Data_NOFLAG.csv'
df = pd.read_csv(csv_path)

# Remove year columns except Y2023
year_cols = [c for c in df.columns if c.startswith('Y') and c != 'Y2023']
df = df.drop(columns=year_cols)

# Filter to only food supply element
df = df[df['Element'] == 'Food supply (kcal/capita/day)']

# Identify items with total Y2023 < 30 across all countries (Filter out bottom 1%)
df = df.fillna(0) # Fill NaNs with 0 for summation
items_to_drop = (
    df.groupby("Item")["Y2023"]
      .sum()
      .loc[lambda x: x < 30]
      .index
)
df_filtered = df[~df["Item"].isin(items_to_drop)]

print('Initial Rows:', len(df), ', Filtered Rows:', len(df_filtered))
print("Initial Unique Item Count:", df["Item"].nunique(), ", Filtered Unique Item Count:", df_filtered["Item"].nunique())

Initial Rows: 65421 , Filtered Rows: 56285
Initial Unique Item Count: 435 , Filtered Unique Item Count: 327


In [95]:
# Load category map for item aggregation
with open("category_map.json", 'r', encoding='utf-8') as f:
    category_map = json.load(f)

# Quick Sanity check
eggs_subcategories = category_map.get("Eggs", [])
print("Eggs Subcategories:", eggs_subcategories)

Eggs Subcategories: ['Eggs from other birds in shell, fresh, n.e.c.', 'Eggs, dried', 'Eggs, liquid', 'Hen eggs in shell, fresh']


In [96]:
# Group by country (Area) to sum sub-items per country
new_rows = []

# Group by country (Area) to sum sub-items per country
for area, group in df.groupby("Area"):
    for category, subitems in category_map.items():
        # Filter the group's rows that match any subitem in this category
        matching_rows = group[group["Item"].isin(subitems)]
        
        # Sum the Y2023 values
        total_y2023 = matching_rows["Y2023"].sum()
        
        # Only add a row if there is a non-zero sum
        if total_y2023 != 0:
            new_rows.append({
                "Area": area,
                "Item": category,
                "Y2023": total_y2023
            })
df_categories = pd.DataFrame(new_rows)

# Show the first few rows
df_categories.head()

,Area,Item,Y2023
0,Afghanistan,Spirits,0.06
1,Afghanistan,Milks - Fluid & Raw,74.26
2,Afghanistan,Milks - Powdered & Condensed,2.61
3,Afghanistan,Butter & Ghee,21.36
4,Afghanistan,Cheese,4.75


In [ ]:

# Original dataset
print(df_filtered[df_filtered["Area"] == "Afghanistan"]["Y2023"].sum())

# Aggregated dataset (Still has same sum)
print(df_categories[df_categories["Area"] == "Afghanistan"]["Y2023"].sum())

# sum of food supply per country in original dataset
df_categories = df_categories.groupby('Area', as_index=False)['Y2023'].sum()
print(df_categories.mean())
df_categories.head()

2307.91
2307.91
Y2023    2914.684272
dtype: float64


,Area,Y2023
0,Afghanistan,2307.91
1,Africa,2533.64
2,Albania,3219.77
3,Algeria,3387.72
4,Americas,3401.60
